# Multi-Commodity Network Flow

This notebook contains code for solving Multi-Commodity Network Flow (MCNF) via Danzig-Wolfe Decomposition. The algorithm is tested on Sioux Falls Network and the solver I used is Open Source solver **ortools**.

**Acknowledgement**: The implementation of the DW decomposition algorithm  is based on the work from [this GitHub repository](https://github.com/yuzhenfeng2002/Multi-Commodity-Network-Flow/) which is implemented via Gurobi. My sincere thanks 💗 to the original author for his clear and insightful code.

In [21]:
import heapq
import time
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

class Node:
    """
    Node in the graph
    """
    def __init__(self, id: int, x=None, y=None) -> None:
        self.id = id
        ## Position
        self.x = x
        self.y = y
        ## Relationship with other nodes
        self.succ = {} # successors in the graph
        self.pred = {} # predecessors in the graph
        self.label = np.inf # for Dijkstra algorithm
        self.spPred = None # also for Dijkstra algorithm
    def __str__(self) -> str:
        return str(self.id)
    def __repr__(self) -> str:
        return str(self.id)
    def __lt__(self, other):
        return self.id < other.id
    
    ## Add successor or predecessor
    def addSucc(self, eId: int, succId: int):
        self.succ[succId] = eId
    def addPred(self, eId: int, predId: int):
        self.pred[predId] = eId

class Edge:
    """
    Edge in the graph
    """
    def __init__(self, id: int, u: Node, v: Node, weight: float, capacity: float = np.inf) -> None:
        self.id = id
        ## The edge is from Node u to Node v
        self.u = u
        self.u.addSucc(id, v.id)
        self.v = v
        self.v.addPred(id, u.id)

        self.capacity = capacity # capacity
        self.weight = weight # weight, may change in the future
        self.initWeight = weight # record the initial weight
        self.capacityUsed = 0 # capacity that has been used by the current flow
        self.capacityLeft = capacity # capacity not used
    
    def __str__(self) -> str:
        return "{:.0f}: {:.0f}->{:.0f} ({:.2f}, {:.2f})".format(self.id, self.u.id, self.v.id, self.initWeight, self.capacity)
    def __repr__(self) -> str:
        return "{:.0f}: {:.0f}->{:.0f} ({:.2f}, {:.2f})".format(self.id, self.u.id, self.v.id, self.initWeight, self.capacity)
    
    ## Update capacity by using a given quantity
    def useCapacity(self, capacityUsed: float):
        self.capacityUsed += capacityUsed
        self.capacityLeft = self.capacityLeft - capacityUsed

class Demand:
    """
    Demand in the graph
    """
    def __init__(self, id: int, o: Node, d: Node, quantity: float) -> None:
        self.id = id
        self.o = o
        self.d = d
        self.quantity = quantity # the quantity of the flow
        self.routes = {} # store the resulting routes of the MCNF problem
    
    ## Hash the route to a string
    def hashRoute(self, route: list):
        s = ""
        for node in route:
            node: Node
            s += str(node.id) + "->"
        return s

    ## Update the dict of the routes, where route is a list of Nodes and ratio is the ratio of flow on the route
    def updateRoute(self, route: list, ratio: float):
        self.routes[self.hashRoute(route)] = self.routes.setdefault(self.hashRoute(route), 0) + ratio

class Network:
    """
    The network
    """
    def __init__(self, nodes={}, edges={}, demands={}) -> None:
        self.nodes: dict = nodes
        self.edges: dict = edges
        self.demands: dict = demands
        self.edgeDict: dict = {} # map the pair of nodes' id to the edge's id
        for edgeId, edge in self.edges.items():
            edge: Edge
            u = edge.u
            v = edge.v
            self.edgeDict[(u.id, v.id)] = edge.id

    ## Add a node
    def addNode(self, id: int, x=None, y=None):
        node = Node(id, x, y)
        self.nodes[id] = node
    
    ## Add an edge, if the end nodes of the edge do not exit, it will add them
    def addEdge(self, id: int, uid: int, vid: int, weight: float, capacity: float = np.inf):
        if uid not in self.nodes:
            self.addNode(uid)
        if vid not in self.nodes:
            self.addNode(vid)
        u = self.nodes[uid]
        v = self.nodes[vid]
        edge = Edge(id, u, v, weight, capacity)
        self.edges[id] = edge
        self.edgeDict[(uid, vid)] = id
    
    ## Add a demand
    def addDemand(self, id: int, oid: int, did: int, quantity: float = 0):
        o = self.nodes[oid]
        d = self.nodes[did]
        demand = Demand(id, o, d, quantity)
        self.demands[id] = demand
    
    ## Load network and demand from a file, example file shown in ./network/*
    def loadNetwork(self, networkFileName: str = "../Assets/Network/SiouxFall/SiouxFalls_net.csv", demandFileName: str = "../Assets/Network/SiouxFall/SiouxFalls_trips.csv"):
        network = pd.read_csv(networkFileName, sep='\t')
        networkDf = network.to_dict("index")
        for edgeId, edgeInfo in networkDf.items():
            self.addEdge(edgeId, edgeInfo['init_node'], edgeInfo['term_node'], edgeInfo['length'], edgeInfo['capacity'] * 2)
        demand = pd.read_csv(demandFileName, sep='\t')
        demandDf = demand.to_dict("index")
        for demandId, demandInfo in demandDf.items():
            if demandInfo["demand"] > 1e-10:
                self.addDemand(demandId, demandInfo["init_node"], demandInfo["term_node"], demandInfo["demand"])

    ## Reset the labels and spPred of all nodes (for new executions of shortest path algorithm)
    def resetNodeLabel(self):
        for node in self.nodes.values():
            node: Node
            node.label = np.inf
            node.spPred = None
    
    ## Reset the capacity of all edges to the initial state
    def resetEdgeCapacity(self):
        for edge in self.edges.values():
            edge: Edge
            edge.capacityUsed = 0
            edge.capacityLeft = edge.capacity
    
    ## Reset the weight of all edges to the initial value
    def resetEdgeWeight(self):
        for edge in self.edges.values():
            edge: Edge
            edge.weight = edge.initWeight

    ## Dijkstra algorithm, return the list of Nodes from u to v and the shortest distance
    def dijkstra(self, u: Node, v: Node = None):
        # Label 就是用来标记找最短路的这个过程：
        # 记录每一次Dijkstra前向搜索时的最短路线的权重
        # 用spPred (Shortest Path Predecessors) 属性来记录最短路线的前向节点
        self.resetNodeLabel()
        self.nodes[u.id].label = 0
        self.nodes[u.id].spPred = None
        minHeap = [(u.label, u)]
        while minHeap:
            currentNode = heapq.heappop(minHeap)[1]
            currentLabel = currentNode.label
            if v != None:
                if v.id == currentNode.id:
                    path = [v]
                    spPred: Node = v.spPred
                    while spPred != None:
                        path.append(spPred)
                        spPred = spPred.spPred
                    path.reverse()
                    return path, v.label
            for nodeId, edgeId in currentNode.succ.items():
                succNode: Node = self.nodes[nodeId]
                succEdge: Edge = self.edges[edgeId]
                if currentLabel + succEdge.weight < succNode.label:
                    succNode.label = currentLabel + succEdge.weight
                    succNode.spPred = currentNode
                    heapq.heappush(minHeap, (succNode.label, succNode))
    
    ## For each demand, assign all the flow to the current shortest path to generate an extreme solution (i.e., a column in DW formulation)
    def generatePathForDemand(self):
        # 备注：该函数不涉及求解器的部分
        # 用途：用于给每个Demand (商品流) 求解一个最短路
        # 注意：这个网络的边权每次都会更新；
        # the information of the extreme solution: 储存该极点的信息
        solution = {"routes": {}}
        # total travel cost of the flow of the solution: 总成本（通行成本）
        totalCost = 0
        # reduced cost of the column: 检验数
        reducedCost = 0
        ## For each demand, assign all the flow to the current shortest path: 对于每一个OD对，计算在调整权重后的路网上的最短路径，并分配流量
        for demandId, demand in self.demands.items():
            demand: Demand
            # calculate the shortest path: 得到最短路径
            sp, _ = self.dijkstra(demand.o, demand.d)
            # the information of the extreme solution -- route: 极点信息——路径
            solution["routes"][demandId] = sp
            # add the total travel cost and the reduced cost according to the path: 根据路径计算总成本和检验数
            lastNode = demand.o
            for node in sp[1: ]:
                node: Node
                # find the edge according to the id of end nodes: 根据路径列表中的点找到对应的边
                edgeId = self.edgeDict[(lastNode.id, node.id)]
                edge: Edge = self.edges[edgeId]
                # update the capacity of the edge: 改变边上的流量
                edge.useCapacity(demand.quantity)
                # total travel cost: 总成本
                totalCost += demand.quantity * edge.initWeight
                # reduced cost: 检验数
                reducedCost += demand.quantity * edge.weight
                lastNode = node
        # the information of the extreme solution: 极点信息
        solution["cost"] = totalCost # total travel cost: 总成本
        solution["reducedCost"] = reducedCost # reduced cost: 检验数（只有这一次迭代有用）
        solution["flow"] = {} # flow in each edge: 各边上的流量
        ## Calculate flow in each edge: 计算各边上的流量
        for edgeId, edge in self.edges.items():
            edge: Edge
            solution["flow"][edgeId] = edge.capacityUsed
        ## Return the extreme solution as well as the information: 返回新极点（及其信息）
        return solution

    def subproblem(self, dualVars: list):
        # Set the weight of all edges according to the dual: 调整路网上各边的权重
        i = 0
        # self.boundedEdges 是一个字典，但 items() 在现代 Python 中保证插入顺序。
        # 我们最好在初始化时就固定好顺序，例如转成一个 list。
        # 假设 self.boundedEdges 的迭代顺序与 self.capacity_constraints 的顺序是一致的。
        for edgeId, edge in self.boundedEdges.items():
            edge: Edge
            # dualVars[i] 是第 i 个容量约束的对偶变量 π_i
            # 新成本 c' = c - π
            edge.weight = edge.initWeight - dualVars[i]
            i += 1
            
        ## Reset the capacity of all edges: 重置图上各边的流量
        self.resetEdgeCapacity()
        
        ## Obtain a new extreme solution...
        # generatePathForDemand 使用新的 edge.weight 计算最短路
        solution = self.generatePathForDemand()
        
        ## Calculate the reduced cost (add dual associated with sum(lambda)=1): 计算检验数
        # 此时，i 等于容量约束的总数。
        # 所以 dualVars[i] 正是我们列表中的最后一个元素，即凸性约束的对偶 σ。
        # solution["reducedCost"] 是 ∑(c' * x) = ∑((c - π) * x)
        # which equals:
        reducedCost = dualVars[i] - solution["reducedCost"] # 更标准的写法
        
        # 你的代码寻找检验数 > 0 的列。
        if reducedCost > 0:
            self.solutions.append(solution)
            
        ## Return the reduced cost: 返回检验数
        return reducedCost
    
    def retrieveSol(self, output=0, outputFilefolder="./output/"):
        ## Reset capacity and weight of all edges
        self.resetEdgeCapacity()
        self.resetEdgeWeight()
        
        ## Obtain the lambda values
        lams = {}
        
        # Gurobi: for v in self.objModel.getVars(): ...
        # OR-Tools: We iterate through the list of lambda variables we've been maintaining.
        for i, lam_var in enumerate(self.lams_vars):
            # lam_var is a pywraplp.Variable object
            if lam_var.solution_value() > 1e-9: # Compare with a small tolerance, not 0
                # The index 'i' corresponds to the solution index, e.g., self.solutions[i]
                lams[i] = lam_var.solution_value()
                
        ## The rest of the logic is identical, as it only depends on the `lams` dictionary
        ## which we have just correctly reconstructed.
        
        ## For each demand, obtain its route(s) and the ratio of flow on the route according to lambda
        for solId, ratio in lams.items():
            solution = self.solutions[solId]
            routesDict: dict = solution["routes"]
            for demandId, route in routesDict.items():
                demand: Demand = self.demands[demandId]
                demand.updateRoute(route, ratio)
                lastNode: Node = demand.o
                for node in route[1:]:
                    node: Node
                    edgeId = self.edgeDict[(lastNode.id, node.id)]
                    edge: Edge = self.edges[edgeId]
                    edge.useCapacity(demand.quantity * ratio)
                    lastNode = node
                    
        ## Output (This part doesn't need any changes)
        routeFile = open(outputFilefolder+"routes_ortools.txt", 'w+')
        routeFile.write("id\tratio\troute\n")
        edgeFile = open(outputFilefolder+"flow_ortools.txt", 'w+')
        edgeFile.write("id\tuid\tvid\tflow\tcapacity\n")
        for demand in self.demands.values():
            demand: Demand
            for route, ratio in demand.routes.items():
                if output: print("{:.0f}\t{:.6f}\t".format(demand.id, ratio) + route)
                else: routeFile.write("{:.0f}\t{:.6f}\t".format(demand.id, ratio) + route + '\n')
        for edge in self.edges.values():
            edge: Edge
            if output: print("{:.0f}\t{:.1f}/{:.1f}".format(edge.id, edge.capacityUsed, edge.capacity))
            else: edgeFile.write("{:.0f}\t{:.0f}\t{:.0f}\t{:.1f}\t{:.1f}\n".format(edge.id, edge.u.id, edge.v.id, edge.capacityUsed, edge.capacity))
        routeFile.close()
        edgeFile.close()
    
    def dwDecomposition(self, M = 1e6, epsilon = 1e-6, output = 0, outputFilefolder="./output/MCNF/"):
        self.solutions = []
        solution = self.generatePathForDemand()
        self.solutions.append(solution)
        # 找到所有有容量上界的边
        self.boundedEdges = {}
        for edgeId, edge in self.edges.items():
            # edge: Edge
            if edge.capacity < np.inf:
                self.boundedEdges[edgeId] = edge
        
        # M = 1e6 # Big-M value from your original code
        # initial_solution = self.solutions[0]
        # bounded_edges_list = list(self.boundedEdges.items()) # 将字典转为列表，保证迭代顺序
        # num_bounded_edges = len(bounded_edges_list)

        # 1. 创建求解器实例
        # Gurobi: masterProblem = gp.Model()
        masterProblem = pywraplp.Solver.CreateSolver('CLP_LINEAR_PROGRAMMING')
        if not masterProblem:
            print("GLOP solver not available.")
            return

        # 2. 创建变量
        # Gurobi: lams = masterProblem.addVar(...)
        lams = masterProblem.NumVar(0.0, 1.0, 'lam[0]')
        self.lams_vars = [lams] # NOTE: 维护一个 lambda 变量的列表，方便后续使用

        # Gurobi: slacks = masterProblem.addVars(...)
        slacks = [masterProblem.NumVar(0.0, masterProblem.infinity(), f's_{j}') for j in range(len(self.boundedEdges))]
        self.slacks_vars = slacks

        # Gurobi: surpluses = masterProblem.addVars(...)
        surpluses = [masterProblem.NumVar(0.0, masterProblem.infinity(), f'a_{j}') for j in range(len(self.boundedEdges))]
        self.surpluses_vars = surpluses

        # 3. 定义目标函数
        # Gurobi: obj=... in addVar/addVars
        objective = masterProblem.Objective()
        objective.SetCoefficient(lams, self.solutions[0]["cost"])
        for j in range(len(self.boundedEdges)):
            # 松弛变量 's' 的目标系数为 0，是默认值，无需设置
            # objective.SetCoefficient(slacks[j], 0) 
            objective.SetCoefficient(surpluses[j], M)
        objective.SetMinimization()

        # 4. 定义约束
        # Gurobi: masterProblem.addConstrs(...)
        # 我们需要将 Gurobi 简洁的生成器表达式，拆解成一个循环
        self.capacity_constraints = []
        for j, k in enumerate(self.boundedEdges.keys()):
            # 创建一个约束，其上下界都是 capacity，即一个等式约束
            # 表达式: flow * lams + 1*s[j] - 1*a[j] = capacity
            capacity = self.boundedEdges[k].capacity
            constraint = masterProblem.Constraint(capacity, capacity, f'capacity_{k}')
            
            # 设置约束中每个变量的系数
            constraint.SetCoefficient(lams, self.solutions[0]["flow"][k])
            constraint.SetCoefficient(slacks[j], 1.0)
            constraint.SetCoefficient(surpluses[j], -1.0)
            self.capacity_constraints.append(constraint) # 保存约束对象，方便后续获取对偶值

        # Gurobi: masterProblem.addConstr(lams == 1)
        self.conv_constraint = masterProblem.Constraint(1.0, 1.0, 'convexity')
        self.conv_constraint.SetCoefficient(lams, 1.0)
        
        startTime = time.time()
        # Prepare to solve!
        
        masterProblem.Solve()
        dual_vars = []
        for constr in self.capacity_constraints:
            # Obtain dual values for capacity constraints
            dual_vars.append(constr.dual_value())
            
        # Second, get the dual for the convexity constraint.
        # This corresponds to the "\sigma" value.
        # 获取极向量的dual variable(只有一个约束所以只有一个值)
        dual_vars.append(self.conv_constraint.dual_value())

        # Start the iteration
        iterNum = 0
        # Solve the Subproblem. The subproblem function itself does not need
        # to change, because it just expects a list of numbers!
        reducedCost = self.subproblem(dual_vars)
        
        objective_value = masterProblem.Objective().Value()
        iterationFile = open(outputFilefolder+"iterations_ortools.txt", 'w+')
        iterationFile.write("iter_num\tobj\treduced_cost\n")
        if output: 
            print("{:.0f}\t\t\t{:.2f}\t\t\t{:.2f}".format(iterNum, objective_value, reducedCost))
        else: 
            iterationFile.write("{:.0f}\t{:.2f}\t{:.2f}\n".format(iterNum, objective_value, reducedCost))
        
        while reducedCost > epsilon and iterNum < 2000:
            s = self.solutions[-1]
            colCoeff = [s["flow"][k] for k in self.boundedEdges.keys()]
            colCoeff.append(1) # 别忘了lambda对应的系数1
            
            # 1. 创建新的 lambda 变量
            # Gurobi 在 addVar 时隐式完成了这一步。OR-Tools 需要显式创建。
            new_lam_name = f"lam[{iterNum+1}]"
            new_lam_var = masterProblem.NumVar(0.0, 1.0, new_lam_name)

            # NOTE：将新创建的变量添加到我们手动维护的列表中，以便后续使用（如最终结果打印）
            self.lams_vars.append(new_lam_var)

            # 2. 在目标函数中设置新变量的成本
            # Gurobi: obj=s["cost"]
            masterProblem.Objective().SetCoefficient(new_lam_var, s["cost"])

            # 3. 将新变量添加到现有的约束中
            # Gurobi: column = gp.Column(colCoeff, masterProblem.getConstrs())
            # OR-Tools: 我们需要遍历已保存的约束，并为它们设置新变量的系数

            # 3.1 更新容量约束
            # 我们假设 self.boundedEdges.keys() 和 self.capacity_constraints 的顺序是一致的
            # 这是因为我们在初始化 RMP 时是按照这个顺序创建约束的
            for i, k in enumerate(self.boundedEdges.keys()):
                flow_on_edge = s["flow"][k]
                # 如果流量为0，理论上可以不设置，但为了清晰起见，全部设置也无妨
                # if flow_on_edge != 0:
                #     self.capacity_constraints[i].SetCoefficient(new_lam_var, flow_on_edge)
                self.capacity_constraints[i].SetCoefficient(new_lam_var, flow_on_edge)

            # 3.2 更新凸性约束 (sum(lams) = 1)
            # 新的 lambda 变量在这个约束中的系数永远是 1
            self.conv_constraint.SetCoefficient(new_lam_var, 1.0)
            # --- 添加列的逻辑到此结束 ---
            
            # Gurobi: masterProblem.optimize()
            # OR-Tools: 求解更新后的 RMP
            status = masterProblem.Solve()
            
            # 检查求解状态，如果失败则中断循环
            if status != pywraplp.Solver.OPTIMAL:
                print(f"Master problem could not be solved to optimality in iteration {iterNum+1}. Status: {status}")
                break
            # OR-Tools: 获取新的对偶变量
            dualVars = []
            for constr in self.capacity_constraints:
                dualVars.append(constr.dual_value())
            # print(len(self.capacity_constraints)) # Should be 76! 
            dualVars.append(self.conv_constraint.dual_value())

            # Gurobi: reducedCost = self.subproblem(dualVars)
            # OR-Tools: 逻辑相同，用新的对偶变量求解子问题
            reducedCost = self.subproblem(dualVars)

            # 更新迭代计数器
            iterNum += 1

            # Gurobi: print("{:.0f}...".format(iterNum, masterProblem.getObjective().getValue(), ...))
            # OR-Tools: 逻辑相同, 获取新的目标值并输出
            objective_value = masterProblem.Objective().Value()
            if output: 
                print("{:.0f}\t\t\t{:.2f}\t\t\t{:.2f}".format(iterNum, objective_value, reducedCost))
            else: 
                iterationFile.write("{:.0f}\t{:.2f}\t{:.2f}\n".format(iterNum, objective_value, reducedCost))
            # print(f"Obj. {objective_value:.2f}")
        
        endTime = time.time() # This line is platform-agnostic, no change needed

        # Gurobi: self.objModel = masterProblem
        # OR-Tools: No need to save the model itself, we have saved the variables.
        # The solver object `masterProblem` will be discarded when the method ends.

        iterationFile.close()

        # Gurobi: masterProblem.getObjective().getValue()
        # OR-Tools: masterProblem.Objective().Value()
        final_objective_value = masterProblem.Objective().Value()
        # Note: You can't use `startTime` here unless you define it at the beginning of the method.
        # Let's assume you've set `startTime = time.time()` before the loop starts.
        print("Iteration time: {:.2f}s. Objective: {:.2f}.".format(endTime - startTime, final_objective_value))

        # --- Writing variables to file ---
        varFile = open(outputFilefolder+"variables_ortools.txt", 'w+')
        varFile.write("var_name\tvalue\n")

        # Gurobi: for v in self.objModel.getVars(): ...
        # OR-Tools: We iterate through our saved lists of variables.

        # 1. Lambda variables
        for lam_var in self.lams_vars:
            var_value = lam_var.solution_value()
            if var_value > 1e-9: # Only write non-zero values
                var_name = lam_var.name()
                if output: print(var_name + '\t' + str(var_value))
                else: varFile.write("{:}\t{:.6f}".format(var_name, var_value) + '\n')

        # 2. Slack variables (slacks_vars)
        # You need to save this list during RMP initialization, just like lams_vars.
        # Let's assume you've already done that: self.slacks_vars = slacks
        for slack_var in self.slacks_vars:
            var_value = slack_var.solution_value()
            if var_value > 1e-9:
                var_name = slack_var.name()
                if output: print(var_name + '\t' + str(var_value))
                else: varFile.write("{:}\t{:.6f}".format(var_name, var_value) + '\n')

        # 3. Surplus variables (surpluses_vars)
        # Let's assume you've already done that: self.surpluses_vars = surpluses
        for surplus_var in self.surpluses_vars:
            var_value = surplus_var.solution_value()
            if var_value > 1e-9:
                var_name = surplus_var.name()
                if output: print(var_name + '\t' + str(var_value))
                else: varFile.write("{:}\t{:.6f}".format(var_name, var_value) + '\n')

        varFile.close()
        # --- End of writing variables ---

        # Process the final solution
        # You should call your new OR-Tools compatible version
        self.retrieveSol(output, outputFilefolder)

if __name__ == "__main__":
    network = Network()
    network.loadNetwork()
    # a, b = network.dijkstra(network.nodes[1], network.nodes[19])
    # test_sol = network.generatePathForDemand()
    network.dwDecomposition()
    

Iteration time: 3.48s. Objective: 3439373.87.
